In [1]:
# imports
import pandas as pd
from functions import *
import requests
from dotenv import dotenv_values
import time

In [2]:
# read in the data
songs = pd.read_csv('data/songs_addon.csv', header=0, index_col=False)

In [3]:
# count the rows
songs.track_id.count()

5565

In [4]:
# show the first rows
songs.head(3)

,title_x,artist_x,track_id,title_y,artist_y,_merge
0,Stranger (feat. Elliphant),Peking Duk,0jVPEBd4wyHupkTHwzQRyc,NaN,NaN,left_only
1,Love$ick (feat. A$AP Rocky),Mura Masa,7Kq16WtEJzdRSNQgdXsn7Y,NaN,NaN,left_only
2,Say It (feat. Tove Lo),Flume,4sQMalN597HdggTu196aL0,NaN,NaN,left_only


In [5]:
# show the last rows
songs.tail(3)

,title_x,artist_x,track_id,title_y,artist_y,_merge
5562,Be My Lover,La Bouche,3YB6nCSuFqNiqOvPtAPK9X,NaN,NaN,left_only
5563,Be About You,Winston Surfshirt,29HSkfe5ITejb0MXhroHtG,NaN,NaN,left_only
5564,All Falls Down,"Alan Walker, Noah Cyrus, Digital Farm Animals,...",0UBLhedidy1DjSF4ncF5zL,NaN,NaN,left_only


In [6]:
# create a list with the track id of each song
songs_list = songs['track_id'].tolist()

In [7]:
# show the first 5
songs_list[0:5]

['0jVPEBd4wyHupkTHwzQRyc',
 '7Kq16WtEJzdRSNQgdXsn7Y',
 '4sQMalN597HdggTu196aL0',
 '2bWrml86uwGVvznX8KKtwx',
 '4rex1k9pBVFaIVKBBssZzi']

In [8]:
# show the last 5
songs_list[5560:5565]

['5MhsZlmKJG6X5kTHkdwC4B',
 '0e9bLw658CErvV0g3byHHR',
 '3YB6nCSuFqNiqOvPtAPK9X',
 '29HSkfe5ITejb0MXhroHtG',
 '0UBLhedidy1DjSF4ncF5zL']

In [14]:
# get authorization token valid for one hour
client_id = dotenv_values('.env')['spotify_id']
client_secret = dotenv_values('.env')['spotify_secret']

data = {
    'grant_type': 'client_credentials',
    'client_id': client_id,
    'client_secret': client_secret,
}
response = requests.post('https://accounts.spotify.com/api/token', data=data)
print(response.json())

{'access_token': 'BQBhnA83o2Fhwq94YsVSnfm6gmmp7Bb4u9GVwqxcvyJF8Mp3c4BjZ-xdpoyaUU8iikNbD6TpC8z11FyvDQm5UMLpElU3IW4QrN03URCek7N-3f-89o4', 'token_type': 'Bearer', 'expires_in': 3600}


In [15]:
# run a test request for one song
token = 'Bearer' + ' ' + 'BQBhnA83o2Fhwq94YsVSnfm6gmmp7Bb4u9GVwqxcvyJF8Mp3c4BjZ-xdpoyaUU8iikNbD6TpC8z11FyvDQm5UMLpElU3IW4QrN03URCek7N-3f-89o4'
headers = {
    'Authorization': token,
}
final_url = 'https://api.spotify.com/v1/audio-features?ids=' + '2nYpbaLykC2cjU8Yt5KF1v'
response = requests.get(final_url, headers=headers)
print(response.json())

{'audio_features': [{'danceability': 0.773, 'energy': 0.623, 'key': 6, 'loudness': -6.173, 'mode': 0, 'speechiness': 0.0377, 'acousticness': 0.22, 'instrumentalness': 0, 'liveness': 0.389, 'valence': 0.714, 'tempo': 129.962, 'type': 'audio_features', 'id': '2nYpbaLykC2cjU8Yt5KF1v', 'uri': 'spotify:track:2nYpbaLykC2cjU8Yt5KF1v', 'track_href': 'https://api.spotify.com/v1/tracks/2nYpbaLykC2cjU8Yt5KF1v', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2nYpbaLykC2cjU8Yt5KF1v', 'duration_ms': 208280, 'time_signature': 4}]}


In [16]:
# run reuqest for 564 songs
url = 'https://api.spotify.com/v1/audio-features?ids='
updated_songs = pd.DataFrame([])
steps = songs_list[5001:5565]
for i in steps:
    url_f = url + i
    r = requests.get(url_f, headers=headers)
    df_temp = r.json()
    df_temp_2 = pd.json_normalize(df_temp, 
                                        sep="_", 
                                        record_path="audio_features", 
                                        meta=[["danceability","energy","key","loudness","mode","speechiness","acousticness","instrumentalness","liveness","valence",
                                        "tempo","duration_ms","time_signature"]], 
                                        errors='ignore')
    updated_songs = pd.concat([updated_songs, df_temp_2], ignore_index=True)
    time.sleep(2)

In [17]:
# count the returned 
updated_songs.id.count()

564

In [18]:
# export the data
updated_songs.to_csv('C:/Users/xxx/DA/neuefische/capstone/data/songs2_06.csv', index=False)